# Apply a model to a video and create new video with detection boxes drawn
<p>
    In the process of the practical we've been wondering how robust our model is on more data and different camera angles. Because we did not intend to label a larger test set we decided to do this by visual inspection.
</p>

In [ ]:
from forward_pass import forward_pass
import numpy as np
import cv2
import matplotlib.pyplot as plt
from fnmatch import fnmatch
import os
import imageio
import shutil

%matplotlib inline

In [ ]:
def empty_directory(folder):
    for the_file in os.listdir(folder):
        file_path = os.path.join(folder, the_file)
        try:
            if os.path.isfile(file_path):
                os.unlink(file_path)
            #elif os.path.isdir(file_path): shutil.rmtree(file_path)
        except Exception as e:
            print(e)

In [ ]:
val_data=False
if val_data:
    data = np.load('val_data.npz')
    img_data, _, lmi = data['imgs'], data['gt'], data['lmi']

In [ ]:
def get_files(path, pattern, not_pattern = None):
    found = []
    for path, subdirs, files in os.walk(path):
        for name in files:
            if fnmatch(name, pattern) and (not_pattern is None or not fnmatch(name, not_pattern)):
                found.append(os.path.join(path, name))
    print("Found %d files"%len(found))
    return found

## Specify video parameters

In [ ]:
cam = 'Cam1'
day = '2018-05-19'
video_folder_id = cam+'/'+day
video_path = '/nfs/students/winter-term-2018/project_2/video_data/videos/'+ video_folder_id
pattern = "*.mp4"
videos = get_files(video_path, pattern, not_pattern='*._*')
videos.sort()

img_data = []
plot_stuff = False
print_every = 20
step_size=1
speedup = 16
picked_videos = range(2000, 2200)
# Some suggestions which I've tried
#picked_videos = range(5000, 6000, step_size)
#found objects in CameraL: [6, 26, 28, 400, 492, 510, 514, 538]
#picked_videos = [400, 492, 510] #mainly beer (CameraL)
#picked_videos = [6, 26, 28]    #jaegermeister, weissbier (CameraL)
#picked_videos = [12, 20, 16]     #williamsbirne, beer (CameraR)
#picked_videos = [46, 58, 64, 66] # (Cam2)
#picked_videos = [2005, 2018, 2064, 2094, 2098] # (Cam1)

In [ ]:
id_str = '_'
for i in [picked_videos[i] for i in [0,-1]]:
    id_str += '%d_'%i
id_str += '_step_size_%d'%step_size

count = 0
for c, i in enumerate(picked_videos):
    video = videos[i]
    images_video = []
    cap = cv2.VideoCapture(video)
    success,image = cap.read()
    count = 0
    while success:
        #cv2.imwrite("frame%d.jpg" % count, image)     # save frame as JPEG file  
        #plt.imshow(image)
        if count % speedup == 0:
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            images_video.append(image)
        success,image = cap.read()
        count += 1
        #print("Extracted %d images"%len(images_video))
    if plot_stuff:
        plt.title("%d"%i)
        plt.imshow(images_video[int(len(images_video)/2)])    
        plt.show()
    else:
        if c % print_every == 0:
            print("Completed loading %d/%d videos"%(c, len(picked_videos)))
    img_data.append(images_video)
img_data = np.vstack(img_data)
print(img_data.shape)

In [ ]:
# Show example image
plt.imshow(img_data[0])
plt.show()

## Pass data to model

In [ ]:
model = '/nfs/students/winter-term-2018/project_2/models/research/object_detection/training_folder_lsml/train/ssd_julius_mobilefpn_large/frozen_inference_graph.pb'
#model = 'train/rfcn/9/frozen_inference_graph.pb'
#model = 'train/pseudo_label_training/all_days/rfcn/1/frozen_inference_graph.pb'

"""
save_dir = '/nfs/students/winter-term-2018/project_2/models/research/object_detection/training_folder_lsml/video_tmp_data/tmp_img/'+video_folder_id+id_str+'_'+str(speedup)+'x'
if os.path.exists(save_dir):
    print("Save directory already exists! Do you want to delete? (yes/no)")
    inp = input()
    assert inp == 'yes', 'Old data will be used then'
    empty_directory(save_dir)
"""
try:
    img_data = forward_pass(model, img_data, gpu='2', BATCH_SIZE=1, thresh=0.5, return_images=True)
except Exception as e:
    print(str(e))

In [ ]:
# Show example image of classified data
#print(np.max(img_data))
#print(np.min(img_data))
img_data = img_data.astype(np.uint8)
plt.imshow(img_data[0])

## Convert result images to mp4 file

In [ ]:
model_name = '_ssd_fpn'
#model_name='_pseudo_rfcn'

In [ ]:
delete_img_afterwards = True
#images = get_files(save_dir, '*.jpg')
#images.sort()
out_dir = '/nfs/students/winter-term-2018/project_2/models/research/object_detection/training_folder_lsml/video_tmp_data/video/'
if not os.path.exists(out_dir):
    os.makedirs(out_dir)
print("Saving to %s"%out_dir)
with imageio.get_writer(out_dir+cam+'-'+day+id_str+model_name+'_'+str(speedup)+'x'+'.mp4', mode='I') as writer:
    for img in img_data:
        #image = imageio.imread(filename)
        #print("Shape: %s\t filename: %s"%(str(image.shape), filename))
        #"""
        try:
            writer.append_data(img)
        except Exception as e:
            print(e)
            #print(image.shape)
            #image = image[:464,:,:]
            #writer.append_data(image)
        #"""
print("Done")
#if delete_img_afterwards:
#    empty_directory(save_dir)
#    print("Deleted images for video generation")

In [ ]:
test = np.copy(img_data[0])
#plt.imshow(test)
#plt.show()
print('start')
try:
    test = cv2.resize(test, (640,640))
    cv2.putText(test, 'TEST', (200, 200-15), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 0, 0),2)
    cv2.rectangle(test, (200,200), (500,500),(0,255,0),8)
    plt.imshow(test)
    plt.show()
except Exception as e:
    print(str(e))
print('done')